In [ ]:
## import os and system packages
import os, sys

## add acolite to path and import
sys.path.append('acolite')
import acolite as ac

In [ ]:
## add proj lib env variable to avoid OGR Error: Corrupt data
## should not be needed, but occurs in certain set ups (like Copernicus JupyterHub)
os.environ['PROJ_LIB'] = "/opt/conda/envs/acolite/share/proj/"

In [ ]:
## load credentials and set as env variables
credentials = ac.shared.import_config('credentials.txt')
for k in credentials: os.environ[k] = credentials[k]

In [ ]:
## location of interest
region_name = 'Albufera'
station_lon =  -0.357
station_lat = 39.3363
date = '2025-11-02'

In [ ]:
## use ACOLITE API to query EarthExplorer
entity_list, identifier_list, dataset_list, metadata_list = ac.api.earthexplorer.query(roi = 'POINT ({} {})'.format(station_lon, station_lat),
                                                                        level = 1, start_date = date,  end_date = date, attributes = True)
## print the results
for scene in identifier_list:
  print(scene)

In [ ]:
## select one WRS-2 tile
tile = '198033'

## check which scenes to get
scene_idx = []
for i, scene in enumerate(identifier_list):
  if tile in scene:
    scene_idx.append(i)

## subset to selected tile
entity_list = [entity_list[i] for i in scene_idx]
identifier_list = [identifier_list[i] for i in scene_idx]
dataset_list = [dataset_list[i] for i in scene_idx]
metadata_list = [metadata_list[i] for i in scene_idx]

In [ ]:
## use ACOLITE API to download the scenes from EarthExplorer
local_scenes = ac.api.earthexplorer.download(entity_list, dataset_list, identifier_list, output = 'Input/')

In [ ]:
# acolite settings
settings = {## basic input and output settings
            ## local_scenes list is used as returned from the API download function
            'inputfile': local_scenes,
            'output': 'Output/',

            ## set up region of interest centred on lon/lat, with box size in km
            'region_name': region_name,
            'station_lon': station_lon,
            'station_lat': station_lat,
            'station_box_size': 7,
            'station_box_units': 'km',

            ## disable ancillary data retrieval, set to True if EarthData credentials are set
            'ancillary_data': False,

            ## use RAdCor processing
            'atmospheric_correction_method': 'radcor',

            ## add a buffer and crop to centre to avoid masked data around the scene edges
            'limit_buffer': 5,
            'limit_buffer_units': 'km',
            'radcor_crop_centre': True,

            ## add the following to enable SWIR based glint correction
            #'dsf_residual_glint_correction': True,
            #'glint_mask_rhos_threshold': 0.10,

            ## delete L1R output NetCDF
            'l1r_delete_netcdf': True,
            ## don't output L1R RGBs
            'l1r_rgb_keys': [],}

In [ ]:
## run acolite with settings dict
ret = ac.acolite.acolite_run(settings)